# Práctica 1

## Preparación del entorno

In [56]:
import numpy as np
import tensorflow as tf
import keras as ker
import matplotlib.pyplot as plt

from keras.preprocessing.image import ImageDataGenerator
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Rescaling




### Variables a definir

In [57]:
# No tocar
num_clases = 10
# La foto es 128x96
xpixel = 128
ypixel = 96
# Tocar
n_neuronas_conv1 = 64
n_neuronas_conv2 = 128
n_neuronas_conv3 = 256
l_rate = 0.15
epoch = 15 
batch = 1 # Realmente en 1 esta bien esto es mas para tiempos de ejecucion con grandes cantidades de datos

# Preprocesamiento de datos

### Esto coge los datos originales y los divide

In [58]:
def meterdatos(batch):
    image_size = (xpixel, ypixel)
    batch_size = batch

    train_generator_t = tf.keras.preprocessing.image_dataset_from_directory(
        directory=r'../Dataset/imgs/train/',
        label_mode = 'categorical',
        validation_split=0.2,
        subset="training",
        shuffle = True,
        seed=1337,
        image_size=image_size,
        batch_size=batch_size,
    )
    train_generator_v = tf.keras.preprocessing.image_dataset_from_directory(
        directory=r'../Dataset/imgs/train/',
        label_mode = 'categorical',
        validation_split=0.2,
        subset="validation",
        shuffle = True,
        seed=1337,
        image_size=image_size,
        batch_size=batch_size,
    )
    return train_generator_t , train_generator_v

#### Augmentation no esta implentado

In [59]:
def meterdatosaug():
        train_datagen = ImageDataGenerator( #Aqui se hacen los cambios a las imagenes
                rescale=1./255,
                shear_range=0.2,
                zoom_range=0.2,
                horizontal_flip=True,
                validation_split = 0.2
                )
        test_datagen = ImageDataGenerator(rescale=1./255)

        train_generator = train_datagen.flow_from_directory(
                directory=r'../Dataset/imgs/train/',
                target_size=(xpixel, ypixel),
                batch_size=batch,
                class_mode='categorical')

        validation_generator = test_datagen.flow_from_directory(
                directory=r'../Dataset/imgs/train/',
                target_size=(xpixel, ypixel),
                batch_size=batch,
                class_mode='categorical')
        return train_generator , validation_generator

### Pintar imagenes dado un tf.keras.preprocessing.image_dataset_from_directory tambien saca el vector de clasificacion

In [60]:
def pintarimg(t):
    plt.figure(figsize=(10, 10))
    for images, labels in t.take(1):
        for i in range(batch):
            plt.imshow(images[i].numpy().astype("uint8"))
            print(labels[i])
            plt.axis("off")

# RED 2 - BUENA DE MOMENTO

In [61]:
def Modelar_red_2(n_conv1,n_conv2,n_conv3,x,y,lr,b):
    model=Sequential()
    # Capa input
    model.add(Rescaling(1./255)) # Como no he rescalado nada usamos esta capa para normalizar los datos
    model.add(Conv2D(filters= n_conv1, kernel_size=(3,3),input_shape=(b, x, y, 3),padding='same',activation='relu')) 
    # Capas convolucionales
    #   >este bloque se puede seguir añadiendo, quiza con menos neuronas, o menos capas convolucionales, pongo dos por dar un ejemplo nada mas
    # (3,3) es mucho se recomienda unsar 1x1 cuando las img no son mayores de 128x128 la nuestra es 128x96
    model.add(Conv2D(n_conv2,(3,3),activation='relu',padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2))) # pooling
    model.add(Dropout(0.2)) # dropout

    model.add(Conv2D(n_conv3,(3,3),activation='relu',padding='same')) 
    model.add(Conv2D(n_conv3,(3,3),activation='relu',padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2))) 
    model.add(Dropout(0.2)) 

    # Capa fully-connected - MLP
    model.add(Flatten())
    model.add(Dense(512,activation='relu')) # red fully-connected
    model.add(Dense(num_clases, activation='softmax')) # capa de salida(softmaxx) 

    model.build((b, x, y, 3))
    print(model.summary()) # con esto inspeccionamos el modelo, muy comodo
    # Compilamos
    adam = tf.keras.optimizers.Adam(learning_rate= lr) # https://keras.io/api/optimizers/adam/ ?
    model.compile(loss="categorical_crossentropy",optimizer=adam,metrics=['accuracy'])
    
    return model

# RED 3 - PROBAR

https://github.com/fomorians/distracted-drivers-keras/blob/master/main.py

In [62]:
def Modelar_red_3(x,y,lr):

    classifier = Sequential()
    model.add(Rescaling(1./255)) # Como no he rescalado nada usamos esta capa para normalizar los datos
# Step 1 - Convolution

    classifier.add(Conv2D(32, (3, 3), input_shape = (xpixel, ypixel, 3), activation = 'relu'))

# Step 2 - Pooling

    classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Step 2(b) - Add 2nd Convolution Layer making it Deep followed by a Pooling Layer

    classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
    classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Step 3 - Flattening

    classifier.add(Flatten())

# Step 4 - Fully Connected Neural Network

# Hidden Layer - Activation Function RELU
    classifier.add(Dense(units = 128, activation = 'relu')) 
# Output Layer - Activation Function Softmax(to clasify multiple classes)
    classifier.add(Dense(units = num_clases, activation = 'softmax'))

    classifier.build((x, y, 3))
    print(classifier.summary()) # con esto inspeccionamos el modelo, muy comodo

    adam = tf.keras.optimizers.Adam(learning_rate= lr) # https://keras.io/api/optimizers/adam/ ?
    classifier.compile(loss="categorical_crossentropy",optimizer=adam,metrics=['accuracy'])
    
    return classifier 

### Y ahora realizamos el fit para entrenar la red

In [63]:
def Entrenar(m,e,v,epo,b):
        # Para parar segun el criterio de la pr
        elcallback = tf.keras.callbacks.EarlyStopping(
          monitor="val_loss",
          min_delta=0.2,
          patience=2,
          verbose=0,
          mode="min",
          baseline=None,
          restore_best_weights=False,
        )
        #(model,entrenamiento,validacion, epoch,batch)
        m.fit(
        e,
        validation_data=v,
        epochs= epo,
        batch_size = b, 
        verbose=1, #Esto te imprime un progress bar con informacion 
        shuffle=True,
        callbacks = elcallback     
        ) 
        return m
# DATO CURIOSO, .FIT() TIENE UNA OPCION LLAMADA VALIDATION_DATA=, IGUAL NI ES NECESARIO CREAR UN GRUPO DE VALIDACION A PARTE

#### Pinta modelo

In [64]:
def pintarmodelo(modelo):
    tf.keras.utils.plot_model(
        modelo,
        to_file='model.png',
        show_shapes=False,
        show_dtype=False,
        show_layer_names=True,
        rankdir='TB',
        expand_nested=False,
        dpi=96,
        layer_range=None,
        show_layer_activations=False
    )

#### Cargar Modelo

In [65]:
def cargarmodelo():
    model = ker.models.load_model('path/to/location') # Carga modelos
    return model

#### Guardar Modelo

In [66]:
def guardarmodelo():
    model.save('ModeloGuardado') # guarda el modelo en la ruta que desees

#### Evaluar Modelo

In [67]:
def Evaluar(m,e_test): # model,entrada_test,salida_test
    return m.evaluate(e_test,verbose=1)

# Campo de Maniobras (Esto seria como el main o algo asi jejejee)

In [1]:
t,v = meterdatos(batch)
model = Modelar_red_3(xpixel,ypixel,l_rate) #Puedes cambiar la funcion para elegir otro modelo



NameError: name 'meterdatos' is not defined

## Para separar outputs de consola

In [ ]:
model = Entrenar(model,t,v,epoch,batch)